In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

from tensorflow import keras
from keras.models import load_model
import pandas as pd
import numpy as np
import os
from keras.preprocessing import image

# import pickle
# import joblib

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
TRAIN_DS_PATH = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/'
SEED = 52
SPLIT = 0.2 
CHECKPOINT_FNAME = 'ckpt_effNet.h5'
LOAD_WEIGHTS = '../input/mlip-ckpt/ckpt_effNet_v3.h5'
TEST_PATH = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/'


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(validation_split=SPLIT, 
                           shear_range=0.2,
                           zoom_range=0.2,
                           horizontal_flip=True,
                           rotation_range=0.1,)
val_datagen = ImageDataGenerator(validation_split=SPLIT,)

train_ds = train_datagen.flow_from_directory(
    TRAIN_DS_PATH, 
    subset='training', 
    seed=SEED,
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE,
)
val_ds = val_datagen.flow_from_directory(
    TRAIN_DS_PATH, 
    subset='validation', 
    seed=SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

In [ ]:
test_ds_path = []
for dirname, _, filenames in os.walk(TEST_PATH):
    path_dict = {}
    for filename in filenames:
        
        path_dict[filename] = os.path.join(dirname, filename)
    test_ds_path.append(path_dict)

In [ ]:
# load model
model = load_model(LOAD_WEIGHTS)

In [ ]:
#load pre trained model
# with open(LOAD_MODEL, 'rb') as file:  
#     Pickled_LR_Model = pickle.load(file)

# Pickled_LR_Model

In [ ]:
# class_indices = {k: v for v, k in enumerate(train_ds.classes)}

In [ ]:
# def get_top5(oneD_list):
#     return np.argpartition(oneD_list,-5)[-5:]

# def listOfTop5(twoD_list):
#     return [get_top5(row) for row in twoD_list]

# key_list = list(class_indices.keys())
# val_list = list(class_indices.values())

# def replaceWithHotelID(oneRow):
#     return [key_list[i] for i in oneRow]

# def replace2DwthHotelID(twoDArray):
#     return [replaceWithHotelID(row) for row in twoDArray]

# #Single Image Prediction Code
# def single_image_prediction(submission_filepath):
    
#         submission = tf.keras.preprocessing.image.load_img(
#             submission_filepath, 
#             grayscale=False, 
#             color_mode="rgb", 
#             target_size=(IMG_SIZE, IMG_SIZE), 
#             interpolation="nearest"
#         )

#         submission_arr = tf.keras.preprocessing.image.img_to_array(submission)
#         submission_arr = np.array([submission_arr])  # Convert single image to a batch.
#         predictions = model.predict(submission_arr)
#         return replace2DwthHotelID(listOfTop5(predictions))

In [ ]:
# prediction_list = []

# for path_dict in test_ds_path:
#     prediction_dict = {}
#     for image, path in path_dict.items():
#         prediction_dict["image_id"] = image
# #         print([int(id) for id in single_image_prediction(path)[0]])
# #         print("single_image_prediction(path)::",single_image_prediction(path))
#         single_image_prediction(path)[0] = [int(id) for id in single_image_prediction(path)[0]]
# #         single_image_prediction(path)[0] = single_image_prediction(path)[0].reverse()
#         pred_list = single_image_prediction(path)[0].copy()
# #         pred_list.reverse()
        
# #         print("pred_list(path)[0]:::", pred_list)
# #         hotel_ids = ' '.join(single_image_prediction(path)[0])
# #         prediction_dict["hotel_id"] = [int(id) for id in single_image_prediction(path)[0]]
# #         prediction_dict["hotel_id"] = [str(list(l)).strip("[]").replace(",","") for l in [int(id) for id in single_image_prediction(path)[0]]]
# #         prediction_dict["hotel_id"] = ' '.join( prediction_dict["hotel_id"])
# #         print("prediction_list::",  *prediction_dict["hotel_id"])
#         prediction_dict["hotel_id"] = [str(list(l)).strip("[]").replace(",", "").replace("'", "") for l in [pred_list]]
#         prediction_list.append(prediction_dict.copy())


# prediction_df = pd.DataFrame(prediction_list)

# # print("prediction_list::",  prediction_list)
# prediction_df["hotel_id"] = prediction_df["hotel_id"].apply(lambda m: str(m).strip("[]").replace("'", ""))
# prediction_df

In [ ]:
#prediction_df.to_csv('submission.csv', index=False)

In [ ]:
submission_json = []
images = []
image_id = []
for img in os.listdir(TEST_PATH):
    image_id.append(img)
    img = os.path.join(TEST_PATH, img)
    img = image.load_img(img, target_size=(IMG_SIZE, IMG_SIZE))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)
images = np.vstack(images)
classes = model.predict(images,)
for i, pred in enumerate(classes):
    final_pred = []
    for p in np.argsort(pred)[::-1][:5]:
#         print(p)
        final_pred.append(list(train_ds.class_indices.items())[p][0])
    submission_json.append({'image_id': image_id[i], 'hotel_id': ' '.join(str(e) for e in final_pred)})

submission_json

In [ ]:
prediction_df = pd.DataFrame(submission_json)
prediction_df["hotel_id"] = prediction_df["hotel_id"].apply(lambda m: str(m).strip("[]").replace("'", ""))
prediction_df

In [ ]:
prediction_df.to_csv('submission.csv', index=False)